# Fluffy recognition model

Trying to build a model that recognises whether an image has something fluffy in it.

## Prepare environment

In [1]:
!pip install fastai==2.5
!pip install -Uqq fastbook
!pip install timm

import fastbook
from fastbook import *
from fastai.vision.all import *
import timm

     |████████████████████████████████| 188 kB 16.3 MB/s 
     |████████████████████████████████| 56 kB 5.8 MB/s 
  Attempting uninstall: fastai
    Found existing installation: fastai 1.0.61
    Uninstalling fastai-1.0.61:
      Successfully uninstalled fastai-1.0.61
     |████████████████████████████████| 720 kB 12.9 MB/s 
     |████████████████████████████████| 46 kB 4.6 MB/s 
     |████████████████████████████████| 1.2 MB 55.6 MB/s 
     |████████████████████████████████| 51 kB 293 kB/s 
     |████████████████████████████████| 376 kB 15.5 MB/s 


In [2]:
#export
from fastai.vision.learner import _add_norm
#export
from timm import create_model
from fastai.vision.learner import _update_first_layer

#exports
def create_timm_body(arch:str, pretrained=True, cut=None, n_in=3):
    "Creates a body from any model in the `timm` library."
    model = create_model(arch, pretrained=pretrained, num_classes=0, global_pool='')
    _update_first_layer(model, n_in, pretrained)
    if cut is None:
        ll = list(enumerate(model.children()))
        cut = next(i for i,o in reversed(ll) if has_pool_type(o))
    if isinstance(cut, int): return nn.Sequential(*list(model.children())[:cut])
    elif callable(cut): return cut(model)
    else: raise NamedError("cut must be either integer or function")

#exports
def create_timm_model(arch:str, n_out, cut=None, pretrained=True, n_in=3, init=nn.init.kaiming_normal_, custom_head=None,
                     concat_pool=True, **kwargs):
    "Create custom architecture using `arch`, `n_in` and `n_out` from the `timm` library"
    body = create_timm_body(arch, pretrained, None, n_in)
    if custom_head is None:
        nf = num_features_model(nn.Sequential(*body.children()))
        head = create_head(nf, n_out, concat_pool=concat_pool, **kwargs)
    else: head = custom_head
    model = nn.Sequential(body, head)
    if init is not None: apply_init(model[1], init)
    return model

#exports
def timm_learner(dls, arch:str, loss_func=None, pretrained=True, cut=None, splitter=None,
                y_range=None, config=None, n_out=None, normalize=True, **kwargs):
    "Build a convnet style learner from `dls` and `arch` using the `timm` library"
    if config is None: config = {}
    if n_out is None: n_out = get_c(dls)
    assert n_out, "`n_out` is not defined, and could not be inferred from data, set `dls.c` or pass `n_out`"
    if y_range is None and 'y_range' in config: y_range = config.pop('y_range')
    model = create_timm_model(arch, n_out, default_split, pretrained, y_range=y_range, **config)
    learn = Learner(dls, model, loss_func=loss_func, splitter=default_split, **kwargs)
    if pretrained: learn.freeze()
    return learn

Download images that I manually collected to train the model on.

In [3]:
IMG_URL = "https://github.com/mihailthebuilder/fluffy-nb/raw/main/fluffy-images.tar.xz"
path = untar_data(IMG_URL)

Check files downloaded and how they're split between fluffy/not fluffy.

In [4]:
file_paths = get_image_files(path)
print(file_paths[:3])

total_files = len(file_paths)
print("total files - "+str(total_files))

def is_fluffy(x): return x[0].islower()

fluffy_files = len([x for x in file_paths if is_fluffy(x.name)])
print("fluffy files - "+str(fluffy_files))

[Path('/root/.fastai/data/fluffy-images/NCFMXCIKQGPTPMMKOIUU.JPEG.jpeg.jpg'), Path('/root/.fastai/data/fluffy-images/obgpwenkqxhdnyconiut.jpg'), Path('/root/.fastai/data/fluffy-images/mnslxqrfucrfrlmyneto.jpg')]
total files - 283
fluffy files - 127


## Establish baseline error rate

The baseline model will always predict that the image is **not** fluffy. So the error rate is the % of images that are fluffy.

In [5]:
fluffy_ratio = fluffy_files / total_files

print("baseline - " + str(round(fluffy_ratio,2)))

baseline - 0.45


## Prepare data

Prepare data for model training

In [6]:
dls = ImageDataLoaders.from_name_func(
    path, file_paths, valid_pct=0.2, seed=42,
    label_func=is_fluffy, item_tfms=Resize(224))

## Train model

In [9]:
learn = timm_learner(dls, 'efficientnetv2_rw_s', metrics=error_rate)
learn.fine_tune(10)

epoch,train_loss,valid_loss,error_rate,time
0,0.906583,0.658549,0.267857,00:04


epoch,train_loss,valid_loss,error_rate,time
0,0.168694,0.370286,0.196429,00:05
1,0.141232,0.239232,0.089286,00:05
2,0.117565,0.233850,0.071429,00:05
3,0.092611,0.241426,0.053571,00:05
4,0.078626,0.213103,0.053571,00:05
5,0.065028,0.215526,0.035714,00:05
6,0.056565,0.257076,0.035714,00:04
7,0.050615,0.259471,0.035714,00:05
8,0.044013,0.231870,0.017857,00:05
9,0.039059,0.182185,0.017857,00:05


The error rate should be somewhere between 3-7%.

## Try out model

Upload your image

In [ ]:
uploader = widgets.FileUpload()
uploader

FileUpload(value={}, description='Upload')

Apply model on image

In [ ]:
img = PILImage.create(uploader.data[0])
fluffy,_,probs = learn.predict(img)
print(f"Is this fluffy?: {fluffy}.")
print(f"Probability it's fluffy: {probs[1].item():.6f}")

Is this fluffy?: True.
Probability it's fluffy: 0.999960


## Previous experiments
- 12.10.2021 - resnet34, 224 pixels - tried different epochs and 2 was the best, with error rates between 5% and 8%
- 12.10.2021 - resnet34 and 500 pixels improved error rates to 3%-6%; 2 epochs still seems the best

## Other notes
- can't use more than 34 layers for resnet architecture together with 500-pixel images as I run out of GPU
- bug relating to file names made the results prior to 12.10.2021 useless
- consider trying EfficientNetV2; it's state of the art and you've already built it for experiments on 11.10.2021